In [5]:
import pandas as pd
import torch as torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import time
import numpy as np
from sklearn.manifold import TSNE

import math
from torch.utils.data import Dataset
import itertools
import torch.nn.functional as F
import matplotlib.pyplot as plt


import gzip
import numpy as np
import random
import scipy
from collections import defaultdict
from scipy.spatial import distance
import dateutil
import os
from sklearn.metrics import roc_auc_score
import json, time
import pandas as pd

def readData_full(path):
    data = []
    for line in open(path):
        d = json.loads(line)
        data.append(d)  
    return data

In [6]:
dataFolder = 'data/ca/'
users_file_name = dataFolder + 'ca_users.json'
places_file_name = dataFolder + 'ca_places.json'
reviews_file_name = dataFolder + 'ca_final_reviews.json'

data_user  = pd.DataFrame(readData_full(users_file_name))
data_places  = pd.DataFrame(readData_full(places_file_name))
data_reviews  = pd.DataFrame(readData_full(reviews_file_name))

In [8]:
st0Min, st0Max = 32.534, 42.009
st1Min, st1Max = -124.4, -114.13
bk = 200
bin0 = np.linspace(st0Min, st0Max, bk)
bin0 = [(a, b) for a, b in zip(bin0[:-1], bin0[1:])]
bin1 = np.linspace(st1Min, st1Max, bk)
bin1 = [(a, b) for a, b in zip(bin1[:-1], bin1[1:])]

def getBin(gps0, gps1):
    b0, b1 = 0, 0
    for i, (a, b) in enumerate(bin0):
        if gps0 >= a and gps0 < b:
            b0 = i
    for i, (a, b) in enumerate(bin1):
        if gps1 >= a and gps1 < b:
            b1 = i
    return bk * b0 + b1

In [9]:
placesDict = {}
for i, p in data_places.iterrows():
    placesDict[p['gPlusPlaceId']] = p
userIDs = {}
itemIDs = {}

interactions = []
interactionsPerUser = defaultdict(list)
userVisitedPlaces = defaultdict(set)
uniquePlaces = set()
for _i, d in data_reviews.iterrows():
    u = d['gPlusUserId']
    i = d['gPlusPlaceId']
    t = d['unixReviewTime']
    r = d['rating']
    cat = str(d['categories'])
    gps = placesDict[d['gPlusPlaceId']]['gps']
    b = getBin(gps[0], gps[1])
    
    uniquePlaces.add(i)
    #dt = dateutil.parser.parse(t)
    #t = int(dt.timestamp())
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((t,u,i,r,b))
    interactionsPerUser[u].append((t,i,r,b))
    userVisitedPlaces[u].add(i)
    
interactions.sort()
userInteractionAvg = sum ([len(interactionsPerUser[u]) for u in interactionsPerUser])  / len(interactionsPerUser)
print(userInteractionAvg)

itemIDs['dummy'] = len(itemIDs)


interactionstrain = []
interactionstest = []
for u in interactionsPerUser:
    interactionsPerUser[u].sort()
    list_users = interactionsPerUser[u]
    lastItem = 'dummy'
    
    for (t,i,r,b) in list_users[:-1]:
        interactionstrain.append((u,i,lastItem,b))
        lastItem = i

    (t,i,r,b) = list_users[-1]
    interactionstest.append((u,i,lastItem,b))
    lastItem = i

itemsPerUser = defaultdict(set)
for u,i,j,b in interactionstrain:
    itemsPerUser[u].add(i)
userIDs = {}
itemIDs = {}

interactions = []
interactionsPerUser = defaultdict(list)
userVisitedPlaces = defaultdict(set)
uniquePlaces = set()
for _i, d in data_reviews.iterrows():
    u = d['gPlusUserId']
    i = d['gPlusPlaceId']
    t = d['unixReviewTime']
    r = d['rating']
    cat = str(d['categories'])
    gps = placesDict[d['gPlusPlaceId']]['gps']
    b = getBin(gps[0], gps[1])
    
    uniquePlaces.add(i)
    #dt = dateutil.parser.parse(t)
    #t = int(dt.timestamp())
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((t,u,i,r,b))
    interactionsPerUser[u].append((t,i,r,b))
    userVisitedPlaces[u].add(i)
    
interactions.sort()
userInteractionAvg = sum ([len(interactionsPerUser[u]) for u in interactionsPerUser])  / len(interactionsPerUser)
print(userInteractionAvg)

itemIDs['dummy'] = len(itemIDs)


interactionstrain = []
interactionstest = []
for u in interactionsPerUser:
    interactionsPerUser[u].sort()
    list_users = interactionsPerUser[u]
    lastItem = 'dummy'
    
    for (t,i,r,b) in list_users[:-1]:
        interactionstrain.append((u,i,lastItem,b))
        lastItem = i

    (t,i,r,b) = list_users[-1]
    interactionstest.append((u,i,lastItem,b))
    lastItem = i

itemsPerUser = defaultdict(set)
for u,i,j,b in interactionstrain:
    itemsPerUser[u].add(i)

nUsers,nItems = len(userIDs),len(itemIDs)
items = list(itemIDs.keys())
print(nUsers,nItems)
print(len(interactionstrain))
print(len(interactionstest))

features_list = ['userId_index', 'placeId_index', 'lastplaceId_index', 'bin']
features_sizes = {
    'userId_index': nUsers,
    'placeId_index':nItems,
    'lastplaceId_index':nItems,
    'bin': bk * bk
}

next_offset = 0
features_offsets={}
index = 0
for k,v in features_sizes.items():
    features_offsets[index] = next_offset
    index += 1
    next_offset += v

17.33063829787234
17.33063829787234
16450 62003
268639
16450


In [16]:
class PopularBaseline():
    def __init__(self):
        placesCount = defaultdict(int)
        for i, r in data_reviews.iterrows():
            placesCount[r['gPlusPlaceId']] += 1
        popularPlaces = sorted([(placesCount[p], p) for p in placesCount], reverse=False)
        self.popScore = [0] * len(itemIDs)
        for i, (c, p) in enumerate(popularPlaces):
            self.popScore[itemIDs[p]] = i
    
    def predict(self, u, i, j):
        return self.popScore[i]

In [17]:
model = PopularBaseline()

In [19]:
interactionsTestPerUser = defaultdict(set)
itemSet = set()
for u,i,j,b in interactionstest:
    interactionsTestPerUser[u].add((i,j,b))
    itemSet.add(i)
    itemSet.add(j)
    
def AUCu(model, u, N):
    win = 0
    positive = [random.sample(interactionsTestPerUser[u],1)[0]] * N
    negative = random.sample(itemSet,N)
    for (i,j,b),k in zip(positive,negative):
        sp = model.predict(userIDs[u], itemIDs[i], itemIDs[j])
        sn = model.predict(userIDs[u], itemIDs[k], itemIDs[j])
        if sp > sn:
            win += 1
    return win/N

def AUC(model):
    av = []
    cnt = 0
    for u in interactionsTestPerUser:
        cnt += 1
        av.append(AUCu(model, u, 10))
    return sum(av) / len(av)
AUC(model)

0.6036109422492312

In [28]:
u = interactions[9999][1]
interactionsPerUser[u][-6:]

[(1321287504.0, '102940239300324391877', 5.0, 6319),
 (1321462308.0, '109648330602459227149', 5.0, 6319),
 (1321891727.0, '104778712706621950626', 5.0, 6319),
 (1325351664.0, '100295524349273441985', 5.0, 740),
 (1327200965.0, '115962894000386189099', 5.0, 6517),
 (1364153036.0, '106689412756761626144', 5.0, 6319)]